In [2]:
import torch
import nemo.collections.asr as nemo_asr 
from pathlib import Path

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from datasets import load_dataset, Audio
from openvoice.api import ToneColorConverter
from melo.api import TTS

BASE_PATH = Path("/mnt/sdg/tzhu/llm")

/home/tzhu/miniconda3/envs/llm/lib/python3.10/site-packages/librosa/util/files.py:10: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_filename
/home/tzhu/miniconda3/envs/llm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:

wav_path = BASE_PATH / "sample.flac"

if not wav_path.exists():
    print("Downloading tiny demo clip (no decoding)…")
    ds = load_dataset("hf-internal-testing/librispeech_asr_dummy", "clean", split="validation")
    ds = ds.cast_column("audio", Audio(decode=False))  # <-- critical: returns file path/bytes, no torchcodec
    rec = ds[0]["audio"]  # {'bytes': b'...', 'path': '1272-128104-0000.flac'}
    with open(wav_path, "wb") as f:
        f.write(rec["bytes"])
    print(f"✅ Saved sample audio to {wav_path}")

# ---------- run NeMo ASR ----------
print("📥 Loading nvidia/canary-1b-flash…")
asr = nemo_asr.models.EncDecMultiTaskModel.from_pretrained("nvidia/canary-1b-flash")

print("🗣️ Transcribing…")
out = asr.transcribe([str(wav_path)], batch_size=1)
transcript = out[0].text if (out and hasattr(out[0], "text")) else out[0]
print("✏️ Transcript:", transcript)

📥 Loading nvidia/canary-1b-flash…
[NeMo I 2025-08-10 00:18:37 mixins:205] _setup_tokenizer: detected an aggregate tokenizer
[NeMo I 2025-08-10 00:18:37 mixins:344] Tokenizer SentencePieceTokenizer initialized with 1152 tokens
[NeMo I 2025-08-10 00:18:37 mixins:344] Tokenizer SentencePieceTokenizer initialized with 1024 tokens
[NeMo I 2025-08-10 00:18:37 mixins:344] Tokenizer SentencePieceTokenizer initialized with 1024 tokens
[NeMo I 2025-08-10 00:18:37 mixins:344] Tokenizer SentencePieceTokenizer initialized with 1024 tokens
[NeMo I 2025-08-10 00:18:37 mixins:344] Tokenizer SentencePieceTokenizer initialized with 1024 tokens
[NeMo I 2025-08-10 00:18:37 aggregate_tokenizer:73] Aggregate vocab size: 5248


[NeMo W 2025-08-10 00:18:37 modelPT:181] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    use_lhotse: true
    input_cfg: null
    tarred_audio_filepaths: null
    manifest_filepath: null
    sample_rate: 16000
    shuffle: true
    num_workers: 8
    pin_memory: true
    prompt_format: canary2
    max_tps: 25
    max_duration: 40.0
    text_field: answer
    lang_field: target_lang
    use_bucketing: true
    bucket_duration_bins:
    - - 3.971
      - 30
    - - 3.971
      - 48
    - - 4.973
      - 37
    - - 4.973
      - 60
    - - 5.85
      - 42
    - - 5.85
      - 71
    - - 6.56
      - 46
    - - 6.56
      - 79
    - - 7.32
      - 49
    - - 7.32
      - 88
    - - 8.19
      - 54
    - - 8.19
      - 99
    - - 8.88
      - 61
    - - 8.88
      - 107
    - - 9.76
      - 66
    - - 9.76
      - 118
    - - 10.56
      - 72
    -

[NeMo I 2025-08-10 00:18:37 features:305] PADDING: 0
[NeMo I 2025-08-10 00:18:48 save_restore_connector:282] Model EncDecMultiTaskModel was successfully restored from /home/tzhu/.cache/huggingface/hub/models--nvidia--canary-1b-flash/snapshots/652da3a37d8bd7e23599359e6c3f857c80cfb657/canary-1b-flash.nemo.


[NeMo W 2025-08-10 00:18:48 dataloader:732] The following configuration keys are ignored by Lhotse dataloader: trim_silence
[NeMo W 2025-08-10 00:18:48 dataloader:479] You are using a non-tarred dataset and requested tokenization during data sampling (pretokenize=True). This will cause the tokenization to happen in the main (GPU) process,possibly impacting the training speed if your tokenizer is very large.If the impact is noticable, set pretokenize=False in dataloader config.(note: that will disable token-per-second filtering and 2D bucketing features)


🗣️ Transcribing…


Transcribing: 1it [00:00,  1.71it/s]

✏️ Transcript: mister Quilter is the apostle of the middle classes, and we are glad to welcome his gospel.


In [4]:
# -------- LLaMA 4 Scout 17B-16E --------
llama_model_id = "meta-llama/Llama-4-Scout-17B-16E-Instruct"

llama_local_path = f"{BASE_PATH}/{llama_model_id}"

llama_tokenizer = AutoTokenizer.from_pretrained(
    llama_local_path, 
    local_files_only=True
)
llama_model = AutoModelForCausalLM.from_pretrained(
    llama_local_path,  
    device_map='auto',
    trust_remote_code=True,
    torch_dtype=torch.float16,
    local_files_only=True  
)
# Test LLaMA
llama_pipe = pipeline("text-generation", model=llama_model, tokenizer=llama_tokenizer)
print("LLaMA test:")
print(llama_pipe("Summarise this candidate's strengths based on the interview notes: ", max_new_tokens=50)[0]["generated_text"])

Loading checkpoint shards: 100%|██████████| 50/50 [01:24<00:00,  1.68s/it]
Device set to use cuda:0


LLaMA test:
Summarise this candidate's strengths based on the interview notes: 1. What is the best strength for the candidate? What makes you think that? 2. What is the best strength for the candidate? What makes you think that? 3. What is the best strength for the candidate? What makes you


In [6]:
# import nltk
# nltk.download('averaged_perceptron_tagger_eng')
# nltk.download('punkt')

ckpt_converter = f"{BASE_PATH}/OpenVoice/checkpoints_v2/converter"
device = "cuda:0" if torch.cuda.is_available() else "cpu"
output_dir = 'outputs_v2'

# Load converter
tone_color_converter = ToneColorConverter(f'{ckpt_converter}/config.json', device=device)
tone_color_converter.load_ckpt(f'{ckpt_converter}/checkpoint.pth')

# Create output directory
Path(output_dir).mkdir(exist_ok=True)

# Load TTS for generating test audio
tts_model = TTS(language='EN', device=device)

# Generate test audio
text = "Hello, this is a test of OpenVoice version two to generate audio."
src_path = f"{output_dir}/test_tts.wav"
tts_model.tts_to_file(text, speaker_id=0, output_path=src_path)
print(f"Generated TTS: {src_path}")


Loaded checkpoint '/mnt/sdg/tzhu/llm/OpenVoice/checkpoints_v2/converter/checkpoint.pth'
missing/unexpected keys: [] []
 > Text split to sentences.
Hello, this is a test of OpenVoice version two to generate audio.
 > ===========================


100%|██████████| 1/1 [00:00<00:00,  7.14it/s]

Generated TTS: outputs_v2/test_tts.wav
